# Overview

The idea is to make a simple app which can be used to create playlists for Spotify which is less annoying. Here is a brief overview of the app.

1. Let the user make new playlists by using songs from existing playlist(s).

We'll be using classes for this. The classes necessary are:
1. The `User` class modeling the user.
2. The `Playlist` class modeling the playlist.
3. The `Song` class modeling the song.

## Attributes and Methods of the Classes

The `User` class has the following attributes:
1. `name`: The name of the user.
2. `id`: The id of the user.
3. `playlists`: A list of `Playlist` objects.
   
The class should also have the following methods:
1. `add_playlist`: Add a new playlist to the user.

The `Playlist` class should have the following attributes:
1. `name`: The name of the playlist.
2. `id`: The id
3. `length`: Number of songs in the playlist.

The class should also have the methods:
1. `add_songs`: Adds new songs.
2. `remove_songs`: Removes songs.

Meanwhile, the `Song` class have the attributes:
1. `name`: The name of the song
2. `id`: id of the song
3. `length`: Duration
4. `artists`: Artist

The class has a single method:
1. `runtime`: Converts the length to mm:ss

# Authentication

In [145]:
import spotipy
from decouple import config

In [154]:
token = spotipy.util.prompt_for_user_token(client_id= config("SPOTIFYID"), client_secret= config("SPOTIFYPASSWORD"), redirect_uri='http://mysite.com/callback/', 
    scope='playlist-read-private user-modify-playback-state playlist-modify-public user-read-recently-played')
sp = spotipy.Spotify(auth=token)

In [147]:
USER = "392uowv8fh00b9ie6ccn9cegs"

# Some Classes

## User

In [90]:
class User:
    def __init__(self, name, id):
        self.name = name
        self.id = id

    def __str__(self):
        return self.name
        
    @property
    def playlists(self):
        playlists = []
        offset = 0
        while True:
            results = sp.current_user_playlists(offset=offset)
            for playlist in results["items"]:
                playlists.append(Playlist(playlist["name"], playlist["uri"], playlist["tracks"]["total"]))
            offset += len(results["items"])
            if offset >= results["total"]:
                break
        return playlists

    def create_playlist(self, name, public = True):
        new_pl = sp.user_playlist_create(self.id, name, public)
        return Playlist(name, new_pl["uri"], 0)

## Song

In [52]:
class Song:
    def __init__(self, name, id, artist, duration):
        self.name = name
        self.id = id
        self.artist = artist
        self.duration = duration
    def __str__(self):
        return self.name + " by " + self.artist

    def __len__(self):
        return self.duration
    @property
    def runtime(self):
        dur_ss = self.duration/1000
        minutes = dur_ss//60
        seconds = dur_ss%60
        return f"{minutes:.0f} Minutes and {seconds:.1f} Seconds"

In [53]:
s = Song("Red", "3267102", "Taylor", 217810)

In [54]:
s.runtime

'3 Minutes and 37.8 Seconds'

## Playlist

In [99]:
class Playlist:
    def __init__(self, name, id, length):
        self.name = name
        self.id = id
        self.length = length
    
    @property
    def songs(self):
        songs = []
        offset = 0
        while True:
            results = sp.playlist_tracks(self.id, offset=offset)
            for track in results["items"]:
                songs.append(Song(track["track"]["name"], track["track"]["id"], track["track"]["artists"][0]["name"], track["track"]["duration_ms"]))
            offset += len(results["items"])
            if offset >= results["total"]:
                break
        return songs

    def __str__(self):
        return self.name

    def add_songs(self, songs):
        if not isinstance(songs, list):
            songs = [songs]
        songs = [song.id for song in songs]
        sp.playlist_add_items(self.id, songs)
        self.length = sp.playlist(self.id)["tracks"]["total"]

    def remove_songs(self, songs):
        if not isinstance(songs, list):
            songs = [songs]
        songs = [song.id for song in songs]
        sp.playlist_remove_all_occurrences_of_items(self.id, songs)
        self.length = sp.playlist(self.id)["tracks"]["total"]

    def __len__(self):
        return self.length
        

In [100]:
hari = User("Hari", "392uowv8fh00b9ie6ccn9cegs")

In [101]:
playlists = hari.playlists
print(playlists[4])

A Tribute to Lata Ji


In [102]:
len(playlists)

60

In [103]:
playlist = playlists[36]
print(playlist, len(playlist), playlist.id)

Best of Lata Mangeshkar 102 spotify:playlist:1QWtxo3TdZcwbem10bG4QL


In [104]:
songs = playlist.songs

In [105]:
len(songs)

102

In [107]:
songs[1].id

'73YTvBoUG34tDnztcdd3iF'

In [106]:
print(songs[10], len(songs[10]))

Chup Gaye Sare Nazare by Lata Mangeshkar 329274


# Using The Classes

## Clone a Playlist

In [63]:
def clone_playlist(playlist, name, user=None):
    if not user:
        user = hari
    new_playlist = hari.create_playlist(name)
    new_playlist.add_songs(playlist.songs)
    return new_playlist

In [64]:
clone_playlist(playlist, "Cloned Playlist")

## Merge Playlists

In [120]:
def merge_playlists(playlists, name, user=None):
    if not user:
        user = hari
    new_playlist = hari.create_playlist(name)
    i = 1
    for playlist in playlists:
        print("Merged Playlist " + str(i) + " of " + str(len(playlists)), end="\r")
        try:
            new_playlist.add_songs(playlist.songs)
        except:
            print("Error Merging Playlist " + str(playlist))
        i += 1
    return new_playlist

In [121]:
bests = []
for p in playlists:
    if "Best" in str(p):
        bests.append(p)

In [122]:
bests

In [123]:
new_pl = merge_playlists(bests, "Merged Playlist")

HTTP Error for POST to https://api.spotify.com/v1/playlists/6Nb9AMRbo7Qh7Lm1R567vu/tracks with Params: {'position': None} returned 400 due to You can add a maximum of 100 tracks per request.


Error Merging Playlist Best of Lata Mangeshkar
Error Merging Playlist Best of Shawn Mendes


In [124]:
len(new_pl)

511

# The App

Let's create a album with every song in the Best of Taylor's Albums.

In [155]:
hari = User("Hari", "392uowv8fh00b9ie6ccn9cegs")

In [156]:
all_playlists = hari.playlists

In [157]:
pls_to_consider=[
    "Best of Red (TV)",
    "Fearless_Bests",
    "Taylor_Swift_Bests",
    "Reputation_Bests",
    "Speak_Now_Bests",
    "1989_Bests",
    "Folklore_Bests",
    "Lover_Bests",
    "Evermore_Bests",
]

In [129]:
pls = []
for pl in all_playlists:
    if str(pl) in pls_to_consider:
        pls.append(pl)

In [130]:
pls

In [131]:
new_pl = merge_playlists(pls, "Bests_From_All_Taylor_Albums")

In [132]:
len(new_pl)

87

In [133]:
all_songs = new_pl.songs

In [134]:
sp.audio_features(all_songs[0].id)

[{'danceability': 0.594,
  'energy': 0.713,
  'key': 9,
  'loudness': -5.314,
  'mode': 1,
  'speechiness': 0.0503,
  'acousticness': 0.000328,
  'instrumentalness': 0,
  'liveness': 0.114,
  'valence': 0.328,
  'tempo': 129.958,
  'type': 'audio_features',
  'id': '6lzc0Al0zfZOIFsFvBS1ki',
  'uri': 'spotify:track:6lzc0Al0zfZOIFsFvBS1ki',
  'track_href': 'https://api.spotify.com/v1/tracks/6lzc0Al0zfZOIFsFvBS1ki',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6lzc0Al0zfZOIFsFvBS1ki',
  'duration_ms': 295413,
  'time_signature': 4}]

In [137]:
print(all_songs[1])

Red (Taylor's Version) by Taylor Swift


In [158]:
for pl in all_playlists:
    if str(pl) == "Temp":
        break

In [159]:
print(pl, len(pl))

Temp 0


In [152]:
pl.remove_songs(pl.songs)